In [1]:
import pandas as pd


def blend_submissions(weight_dict, output_path, target_col):
    dataframes = []

    # Load each submission with its weight
    for path, weight in weight_dict.items():
        df = pd.read_csv(path, usecols=["id", target_col])

        # Add weighted prediction column
        df["weighted_pred"] = df[target_col] * weight

        dataframes.append(df[["id", "weighted_pred"]])

    # Merge all submissions on 'id'
    merged = dataframes[0]

    for df in dataframes[1:]:
        merged = merged.merge(df, on="id", how="inner")

        # Combine weighted predictions
        merged["weighted_pred"] = (
            merged["weighted_pred_x"] + merged["weighted_pred_y"]
        )

        merged.drop(
            columns=["weighted_pred_x", "weighted_pred_y"],
            inplace=True
        )

    # Compute total weight
    total_weight = sum(weight_dict.values())

    # Compute blended prediction
    merged[target_col] = merged["weighted_pred"] / total_weight

    # Prepare final DataFrame
    blended = merged[["id", target_col]]

    # Save blended submission
    blended.to_csv(output_path, index=False)

    print(f"✅ Blended submission saved to {output_path}")


def main():
    TARGET = "Heart Disease"

    weight_dict = {
        "/kaggle/input/datasets/anthonytherrien/heart-disease-prediction-vault/submission.csv": 2.5,
        "/kaggle/input/datasets/anthonytherrien/heart-disease-prediction-vault/submission (1).csv": 0.1,
    }

    blend_submissions(weight_dict, "submission.csv", TARGET)


if __name__ == "__main__":
    main()

✅ Blended submission saved to submission.csv
